In [ ]:
# importing the all neccesaary libraries

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img,img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# Downloading the dataset from Kaggle
# You need to have Kaggle API credentials set up in your environment
# Make sure to install the kagglehub package if you haven't already
import kagglehub

# Download latest version
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path)

# Unzip the dataset



Path to dataset files: C:\Users\suraj\.cache\kagglehub\datasets\masoudnickparvar\brain-tumor-mri-dataset\versions\1


In [ ]:
# Building the model and extracting the images and labels and saving the model

# Constants
IMG_SIZE = (128, 128)
BATCH_SIZE = 64  # Increase if GPU memory allows
EPOCHS = 10
NUM_CLASSES = 4  # glioma, meningioma, pituitary, no_tumor

def load_data(data_dir):
    """Load and preprocess image data"""
    datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        validation_split=0.2,
        rotation_range=10,
        zoom_range=0.1,
        horizontal_flip=True
    )

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, val_generator

def build_model():
    """Build model using MobileNetV2 for fast + accurate training"""
    base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    base_model.trainable = False  # Freeze base layers for speed

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def train_model(data_dir, model_save_path):
    """Train the model and save it"""
    train_gen, val_gen = load_data(data_dir)
    model = build_model()

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=val_gen,
        callbacks=[early_stop]
    )

    model.save(model_save_path)

    # Plot training history
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.legend()

    plt.savefig('training_history.png')
    plt.close()

    return model

if __name__ == "__main__":
    data_directory = "/kaggle/input/brain-tumor-mri-dataset/Training"
    model_path = "models/brain_tumor_classifier.h5"

    os.makedirs("models", exist_ok=True)
    trained_model = train_model(data_directory, model_path)
    print(f"✅ Model trained and saved to {model_path}")
